# Project: PROWPI002 (Project World Population Indicators 002)
# Process: Forecast ETL

## Project libraries

In [55]:
import pandas   as     pd                     # Pandas
from   datetime import datetime, timedelta    # Date & time

## File imports

In [56]:
df_exc01 = pd.read_excel('/content/ISS_param.xls','FORECAST')
df_exc02 = pd.read_xml('/content/XML_FactInternetSales.xml')

## Param transforms

In [57]:
floFct = float(df_exc01.factor)
strAco = str(int(df_exc01.accounting))
strFor = str(int(df_exc01.forecast))

## Function to transform quantity and amounts
## **_Notice_**: Factor has to be an integer because our customer only sells complete bicycles

In [58]:
def ImportQuantity(strOld, floFct):
    strNew = strOld.replace(".","")
    strOld = strNew
    strNew = strOld.replace(",",".")
    floNew = float(strNew) * floFct
    strN01 = str(floNew).replace(".",",")
    return strN01

In [59]:
def YearForecast(strDatOld, strAco):
  strDatYY  = strDatOld[6:10]
  strDatRst = strDatOld[0:6]
  if strAco==strDatYY:
    strDatYY = strFor
  strDatNew = strDatRst + strDatYY
  return strDatNew

## Construction of transformed dataframe

In [ ]:
i = 0

for row in df_exc02.FactInternetSalesPK:
  df_exc02.OrderQuantity.iloc[i] = ImportQuantity(str(df_exc02.OrderQuantity.iloc[i]), floFct)
  df_exc02.ExtendedAmount.iloc[i] = ImportQuantity(str(df_exc02.ExtendedAmount.iloc[i]), floFct)
  df_exc02.SalesAmount.iloc[i] = ImportQuantity(str(df_exc02.SalesAmount.iloc[i]), floFct)
  df_exc02.OrderDate.iloc[i] = YearForecast(df_exc02.OrderDate.iloc[i], strAco)
  df_exc02.ShipDate.iloc[i] = YearForecast(df_exc02.ShipDate.iloc[i], strAco)
  df_exc02.DueDate.iloc[i] = YearForecast(df_exc02.DueDate.iloc[i], strAco)
  i += 1

In [ ]:
i = 0

for row in df_exc02.FactInternetSalesPK:
  strYY = str(df_exc02.FactInternetSalesYear.iloc[i])
  if strYY != strAco:
    df_exc02.drop(i, axis=0, inplace=True)
    df_exc02.reset_index(drop=True, inplace=True)
    print(strYY,i)
    i=i-1
  i += 1

In [ ]:
df_exc02.info()

## Export to CSV to be processed in Pentaho Data Integration

In [64]:
df_exc02.to_csv('/content/CSV_FactInternetSales.CSV', index=False, sep=';')